In [1]:
from abc import ABC, abstractmethod

# class Losowanie:
#     def __init__(self, flag : bool = False):
#         self.flag = flag
#
#     if flag:
#

class Nucleus(ABC):
    def __init__(self,
                 center,
                 axes_size,
                 angle=0,
                 color=(160, 83, 179),
                 thickness=-1,
                 border_color=(107, 26, 121),
                 border_thickness=2):

        self.center = center
        self.axes_size = axes_size
        self.angle = angle
        self.color = color
        self.thickness = thickness
        self.border_color = border_color
        self.border_thickness = border_thickness




In [2]:
from src.functions.poisson_sampling import poisson_sampling


class CancerNucleus(Nucleus):
    _lista_punktów = []
    _licznik = 0
    _liczba_punktów = 0
    def __init__(self, axes_size, irregularity=0.3, angle=0, color=(160, 83, 179),
                 thickness=-1, border_color=(107, 26, 121), border_thickness=2):

        if not CancerNucleus._lista_punktów:
            CancerNucleus._lista_punktów = poisson_sampling(128, 128)

        if CancerNucleus._licznik < len(CancerNucleus._lista_punktów):
            srodek = CancerNucleus._lista_punktów[CancerNucleus._licznik]
            CancerNucleus._licznik += 1
        else:
            raise Exception("Brak punktów środka jądra do przydzielenia")

        super().__init__(
            center = srodek
            , axes_size = axes_size
            , angle = angle
            , color = color
            , thickness = thickness
            , border_color = border_color
            , border_thickness = border_thickness
        )
        self.irregularity = irregularity

    def return_center(self):
        return self.center





In [ ]:
class Losowanie(ABC):
    @abstractmethod
    def losuj(self, zakres_od, zakres_do):
        pass

In [ ]:
class RozkladNormalny(Losowanie):
    def __init__(self, od, do):
        self.od = od
        self.do = do
        def losuj(self, zakres_od, zakres_do):
            # return(

            # )
            pass

In [ ]:
def losowanie_centrum_bez_nakladania(rodzaj_losowania, wymiarxpowierzchni, wymiarypowierzchni, ilosc):
    x = wymiarxpowierzchni
    y = wymiarypowierzchni





In [4]:
c1 = CancerNucleus((10,10))
c1.return_center()


(30.074052476084923, 102.9130245669864)

In [5]:
c2 = CancerNucleus((10,10))
c2.return_center()

(34.570727736960954, 99.02396207816349)